<div style="background-color:#e9eeef; border-right: 10px solid #36b38d ;" height=300 width=100%>
    <font color=#2b3444 size=8 align=left ><strong> Revenue Forecasting</strong></font><br>
    <font color=#36b38d size=6 align=left ><strong> Model Training Interface</strong></font>
</div>
 



In [16]:
import ipywidgets as widgets
import pandas as pd
from ipywidgets import FileUpload
import base64
from IPython.display import HTML
import time
from ipywidgets import IntProgress
from IPython.display import display
import sklearn
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import xgboost as xgb
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
import datetime
import re
import time
import pickle
# import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 10)
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# import xgboost as xgb
# from math import sqrt
# from sklearn.metrics import mean_squared_error
# from scipy.stats import uniform, randint
# from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, RandomizedSearchCV
# from sklearn.preprocessing import StandardScaler
# import pydot_ng as pydot
# from IPython.display import Image
# import math
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix
# from xgboost import XGBRegressor
# from sklearn.preprocessing import StandardScaler
import warnings
# import sys
# import io

# XGBoost Model 
**Predictions can be generated using two models - XGBoost and FBprophet. This dashboard only hosts the XGBoost Model.**

**Steps to use the dashboard are listed below**

_______________________________________________________________________________________________________________________________
## Steps to Generate Predictions
### - Select Pre existing or re-train a new model
**You can choose to generate predictions with the existing model. Or you can choose to re-train the model with a new dataset.**

**For Existing Model: Each prediction cycle takes a single .csv test file and latest version of PD Sync.The test file MUST contain the month and year columns. If you would like to predict for multiple months, please combine all the months into a single file or predict for one month at a time.**

**For example if you would like to generate predictions for 2 months - Nov and Dec. You can either combine the data for both the months in a single file or upload two separate files containing data for Nov and Dec respectively.**

**For Re-training: Each prediction cycle takes three files, pd sync, revenue projection and a test file, all three files can be .csv or .xlsx in format. PD Sync and Revenue Projection files are combined to create the training data and the test file is used to generate predictions based on the newly trained model.**


### - Select level of training / testing
**What are Levels?**

**There are 3 levels within which the model can generate predictions. These are - L0, L1 and L2.**

**L0 - For predictions at Quantiphi level**

**L1 - For predictions at Channel vs. BU level**

**L2 - For further predictions within a Channel and BU.** 
**In channels, only GCP has been considered for predictions due to sufficient data volume. Within GCP predictions can be made for different regions and practices.**

**IN BUs, only BFSI has been considered for predictions due to sufficient data volumne. Within BFSI, predictions can be made for deals that are in AWS, GCP or Direct**

_______________________________________________________________________________________________________________________________

### <font color='red'>Important points before uploading dataset</font>
**For pre-existing data, two files are required - latest PD Sync and a test file**
* Test file should contain only the following columns with the exact spelling & format - Year , Month ,	Deal ID	, Weights , 	Stages , Leads UW , Business Unit , Channel
* PD Sync file can contain any no. of columns

**For re-training data, three files are required - latest PD Sync and a test file**
* Test file should contain only the following columns with the exact spelling & format - Year , Month ,	Deal ID	, Weights , 	Stages , Leads UW , Business Unit , Channel
* PD Sync file and Revenue projection file can contain any no. of columns
_______________________________________________________________________________________________________________________________

In [44]:
warnings.filterwarnings("ignore")

output_box = widgets.Output()

dataset_selector = widgets.RadioButtons(
    options=['Predict with existing model', 'Re-train model'],
    description='Select: ',
    disabled=False
)
        
#######---------------------------------------UPLOADER FUNCTION FOR PRE-EXISTING DATA-----------------------------------########

#Uploader function for pd sync test
label_pdsync_test=widgets.Label('Please upload PD Sync file in .csv or .xlsx format')
uploader_pdsync_test = widgets.FileUpload(multiple=False) 


get_pdsync_test_button= widgets.Button(description='Next',button_style='info',tooltip='Run')


def get_pdsync_test(b):
    input_file = list(uploader_pdsync_test.value.values())[0]
    content = input_file['content']
    #content = io.StringIO(content.decode('utf-8'))
    
    input_file_Name=input_file['metadata']['name']
    File_name_temp1=input_file_Name.split('.')
    File_type=File_name_temp1[1]
    if File_type=='csv':
        df_pdsync_test = pd.read_csv(input_file_Name)
    else:
        df_pdsync_test = pd.read_excel(content)    

#     display(df_pdsync_test)
    get_pdsync_test.data = df_pdsync_test
    
label_test=widgets.Label('Please upload the test file in .xlsx or .csv format')
uploader_test = widgets.FileUpload(multiple=False) 


get_test_button= widgets.Button(description='Next',button_style='info',tooltip='Run')

#Uploader function for test data
def get_test(b):
    input_file = list(uploader_test.value.values())[0]
    content = input_file['content']
    #content = io.StringIO(content.decode('utf-8'))
    
    input_file_Name=input_file['metadata']['name']
    File_name_temp1=input_file_Name.split('.')
    File_type=File_name_temp1[1]
    if File_type=='csv':
        df_test = pd.read_csv(input_file_Name)
    else:
        df_test = pd.read_excel(content)    

#     display(df_test) 
    get_test.data = df_test
#########--------------------------------------------L1 SELECTOR & FORMATTER FOR EXISTING----------------------------------------------------########    
B = widgets.Button(
     description='Format Data',
     disabled=False,
     button_style='info', # 'success', 'info', 'warning', 'danger' or ''
     tooltip='Click me',
     icon='run'
)
def on_change_l1(change):
    with outputlev:
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'GCP':
                with outputlev:
                    print('GCP Selected')
                    display(B)
            elif change['new'] == 'BFSI':
                with outputlev:
                    print('BFSI Selected')
                    display(B)
            elif change['new'] == 'AWS':
                with outputlev:
                    print('AWS Selected')  
                    display(B)
            elif change['new'] == 'TMEG':
                with outputlev:
                    print('TMEG Selected')
                    display(B)
            elif change['new'] == 'HCLS':
                with outputlev:
                    print('HCLS Selected')
                    display(B)
#########--------------------------------------------L2 SELECTOR & FORMATTER EXISTING----------------------------------------------------########
def on_change_l2(change):
    with outputlev:
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'GCP-US-East':
                with outputlev:
                    print('GCP-US-East Region Selected')
                    display(B)
            elif change['new'] == 'GCP-US-West':
                with outputlev:
                    print('GCP-US-West Region Selected')
                    display(B)
            elif change['new'] == 'GCP-US-Central':
                with outputlev:
                    print('GCP-US-Central Region Selected')
                    display(B)        
            elif change['new'] == 'BFSI-AWS':
                with outputlev:
                    print('BFSI-AWS Selected') 
                    display(B)
            elif change['new'] == 'BFSI-Direct':
                with outputlev:
                    print('BFSI-Direct Selected')
                    display(B)
            elif change['new'] == 'BFSI-GCP':
                with outputlev:
                    print('BFSI-GCP Selected')   
                    display(B)
#########--------------------------------------------LEVEL SELECTOR EXISTING----------------------------------------------------########

level_selector = widgets.RadioButtons(
                         options=['L0','L1','L2'],
                         value=None,
                         description='Select Level: ',
                         disabled=False,
#     layout=widgets.Layout(width='100%')
)

level1_selector = widgets.RadioButtons(
                         options=['GCP','AWS','BFSI', 'TMEG', 'HCLS'],
                         value=None,
                         description='Select Level: ',
                         disabled=False,
#     layout=widgets.Layout(width='100%')
)

level2_selector = widgets.RadioButtons(
                         options=['BFSI-AWS', 'BFSI-Direct', 'BFSI-GCP', 'GCP-US-Central','GCP-US-East','GCP-US-West'],
                         value=None,
                         description='Select Level: ',
                         disabled=False,
)


lev_button = widgets.Button(
description='Format Data',
disabled=False,
button_style='info', 
tooltip='Run'
    # #     icon='play'
)

label_level = widgets.Label('Please select level for prediction')

def selectlevel(button):
    with output_box:
        selection = level_selector.get_interact_value()
        if (selection == "L0"):
            with output_box:
                display(l0select)
                time.sleep(10)
                l0_format_test()               
        elif (selection =="L1"):
            with output_box:
                print('L1 Level Selected')
                level1_selector.observe(on_change_l1)
                display(level1_selector)

        elif (selection =="L2"):
            with output_box:
                print('L2 Level Selected')
                level2_selector.observe(on_change_l2)
                display(level2_selector)    

l0_button = widgets.Button(
     description='Format Data',
     disabled=False,
     button_style='info', # 'success', 'info', 'warning', 'danger' or ''
     tooltip='Click me',
     icon='run'
)

def lessthanzero(x):
    return x <= 0 
#########--------------------------------------------L0 FORMATTING TESTING EXISTING----------------------------------------------------########
l0_predict_button= widgets.Button(description='Predict',button_style='info',tooltip='Run')
text = 'Running Predictions for L0'
predictionl0_Widget = widgets.HTML(value = f"<b><font color='#36b38d' size=5>{text}</b>")

def create_download_link_l0( df, title = "Download formatted test CSV file", filename = "formatted-data.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

def create_download_link_l0_final( df, title = "Download L0 prediction CSV file", filename = "prediction-L0.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

def l0_format_test():
#     df_pdsync_test = df_pdsync_test.rename(columns={"Deal ID": "dealid"})
    df_test = get_test.data
    df_pdsync_test = get_pdsync_test.data
    df_pdsync_test = df_pdsync_test.drop(columns=['Weights'], axis=1)
    df_test = df_test.rename(columns={"Deal ID": "dealid"})
    df_all = df_test.merge(df_pdsync_test.drop_duplicates(), on=['dealid','dealid'], 
                    how='left', indicator=True)
    df_all['Leads UW'] = df_all['Leads UW'].str.replace(',', '').str.replace('$', '').astype(int)
    dropstage = df_all['Stages'].isin(['lost', 'On Hold', 'DA', 'Deleted'])
    df_all = df_all[~dropstage]
    df_all = df_all[['Year', 'Month', 'dealid', 'Leads UW', 'dealvalue', 'Stages',
       'Weights', 'region', 'industry',
       'billingtype', 'solcategory', 'projectType', 'businessimpact']]
    dealvalue_eval = df_all.loc[df_all['dealvalue'].apply(lessthanzero)]
    df_all = df_all[~df_all.dealvalue.isin(dealvalue_eval.dealvalue)]
    cols2 = ['Stages','region', 'industry',
        'billingtype', 'solcategory', 'projectType', 'businessimpact']
    df_all[cols2] = df_all[cols2].astype(str)
    label_encode_columns = df_all[cols2].apply(LabelEncoder().fit_transform)
    cols1 = df_all.select_dtypes([np.number]).columns
    numerical = df_all[cols1]
    numerical['tmp'] = 1
    label_encode_columns['tmp'] = 1
    result = pd.concat([numerical, label_encode_columns], axis=1)
    result = result.drop(columns=['tmp'])
    result['Date'] = pd.to_datetime(result[['Year', 'Month']].assign(DAY=1))
    result = result.drop(columns=['Year','Month'])
    result = result[['Date',
      'dealid',
#       'Actuals',
      'Weights',
      'Leads UW',
     'dealvalue',
     'Stages',
#     #  'PSO/Non-PSO',
      'region',
#     #  'Practice',
#     #  'sourceoflead',
       'industry',
       'billingtype',
       'solcategory',
#     #  'Business Category1',
       'projectType',
#     #  'projectextension',
       'businessimpact',
    ]]                                
                                     
    result['Date'] = pd.to_datetime(result['Date'])
    result = result.set_index(result['Date'])
    result = result.sort_index()
    result = result.drop(columns=['Date'])
    result = result.rename(columns={"Stages": "Stage"}) 
    result = result.rename(columns={"region": "Region"})
    result = result.rename(columns={"Weights":"Weight"}) 
    test = result.drop(columns=['dealid'])
    test = test[['Weight', 'dealvalue', 'Leads UW', 'Stage', 'Region', 'industry', 'billingtype', 'solcategory', 'projectType', 'businessimpact']]
#     test = test.apply(pd.to_numeric)
    display('Displaying formatted file below: ')
    display(test)
    display(create_download_link_l0(test))
    display(predictionl0_Widget)
    display("Predicting.Please Wait..")
    time.sleep(10)
    display("Predicting.Please Wait..")
    time.sleep(10)
    display("Predicting.Please Wait..")
    time.sleep(10)
    month = df_test['Month'].iloc[1]
    year = df_test['Year'].iloc[1]
    print("Prediction for {}/{} is : \n".format(month,year))
    display(l0_predict(test))
    pred = l0_predict(test)
   
    df_l0 = pd.DataFrame(data = [[month,year,pred,'nan','nan','nan']], columns=['Month', 'Year', 'Prediction', 'Actuals', 'MAE', 'MAPE'])
    display(create_download_link_l0_final(df_l0))
    
######--------------------------------------------L0 MODEL LOAD TESTING---------------------------------------------------------########    

def l0_model():
    with open('./xgb_L0.pkl', 'rb') as f:
        l0_model = pickle.load(f)
    return l0_model

def l0_predict(df):
         # Feature arragend as per the model
    model_l0=l0_model()
    predict_l0 = model_l0.predict(df)               # Prediction using the model
    prediction = np.sum(predict_l0, dtype=np.int32)      # Predicted result is made into readable format
    return prediction



    
#######-------------------------------------UPLOADER FUNCTION FOR RE-TRAINING DATA--------------------------------------########

#Uploader function for pd sync test
label_pdsync_rt=widgets.Label('Please upload PD Sync file in .csv or .xlsx format')
uploader_pdsync_rt = widgets.FileUpload(multiple=False) 


get_pdsync_rt_button= widgets.Button(description='Next',button_style='info',tooltip='Run')


def get_pdsync_rt(b):
    input_file = list(uploader_pdsync_rt.value.values())[0]
    content = input_file['content']
    #content = io.StringIO(content.decode('utf-8'))
    
    input_file_Name=input_file['metadata']['name']
    File_name_temp1=input_file_Name.split('.')
    File_type=File_name_temp1[1]
    if File_type=='csv':
        df_pdsync_rt = pd.read_csv(input_file_Name)
    else:
        df_pdsync_rt = pd.read_excel(content)    

#     display(df_pdsync_rt)
    get_pdsync_rt.data = df_pdsync_rt
    
    
label_rp=widgets.Label('Please upload the latest rev projections sheet in .xlsx or .csv format')
uploader_rp = widgets.FileUpload(multiple=False) 


get_rp_button= widgets.Button(description='Next',button_style='info',tooltip='Run')

#Uploader function for test data
def get_rp(b):
    input_file = list(uploader_rp.value.values())[0]
    content = input_file['content']
    #content = io.StringIO(content.decode('utf-8'))
    
    input_file_Name=input_file['metadata']['name']
    File_name_temp1=input_file_Name.split('.')
    File_type=File_name_temp1[1]
    if File_type=='csv':
        df_rp= pd.read_csv(input_file_Name)
    else:
        df_rp = pd.read_excel(content)    

#     display(df_rp) 
    get_rp.data = df_rp
label_test_rt=widgets.Label('Please upload the test sheet for which you want to test the re-trained model in .xlsx or .csv format')
uploader_test_rt = widgets.FileUpload(multiple=False) 


get_rt_button= widgets.Button(description='Next',button_style='info',tooltip='Run')

#Uploader function for test data
def get_test_rt(b):
    input_file = list(uploader_test_rt.value.values())[0]
    content = input_file['content']
    #content = io.StringIO(content.decode('utf-8'))
    
    input_file_Name=input_file['metadata']['name']
    File_name_temp1=input_file_Name.split('.')
    File_type=File_name_temp1[1]
    if File_type=='csv':
        df_rt= pd.read_csv(input_file_Name)
    else:
        df_rt = pd.read_excel(content)    
#     display(df_rt)
    get_test_rt.data = df_rt
    
#######----------------------------------------LEVEL SELECTOR RE-TRAIN----------------------------------------------############
level_selector_rt = widgets.RadioButtons(
                         options=['L0','L1','L2'],
                         value=None,
                         description='Select Level: ',
                         disabled=False,
#     layout=widgets.Layout(width='100%')
)

level1_selector_rt = widgets.RadioButtons(
                         options=['GCP','AWS','BFSI', 'TMEG', 'HCLS'],
                         value=None,
                         description='Select Level: ',
                         disabled=False,
#     layout=widgets.Layout(width='100%')
)

level2_selector_rt = widgets.RadioButtons(
                         options=['BFSI-AWS', 'BFSI-Direct', 'BFSI-GCP', 'GCP-US-Central','GCP-US-East','GCP-US-West'],
                         value=None,
                         description='Select Level: ',
                         disabled=False,
)


lev_button_rt = widgets.Button(
description='Format Data',
disabled=False,
button_style='info', 
tooltip='Run'
    # #     icon='play'
)

label_level_rt = widgets.Label('Please select level for prediction')
# l0label = widgets.Label('L0 Level Selected')
text1 = 'L0 Level Selected'
l0select = widgets.HTML(value = f"<b><font color='#36b38d' size=3>{text1}</b>")


# l0start = widgets.Button(
# description='Next',
# disabled=False,
# button_style='info', 
# tooltip='Run'
#     # #     icon='play'
# )
def selectlevel_rt(button):
    with output_box:
        selection = level_selector_rt.get_interact_value()
        if (selection == "L0"):
            with output_box:
                display(l0select)
                time.sleep(10)
                l0_format_rt()                       
        elif (selection =="L1"):
            with output_box:
                print('L1 Level Selected')
#                 level1_selector_rt.observe(on_change_l1_rt)
#                 display(level1_selector_rt)

        elif (selection =="L2"):
            with output_box:
                print('L2 Level Selected')
#                 level2_selector_rt.observe(on_change_l2_rt)
#                 display(level2_selector_rt)    

l0_button_rt = widgets.Button(
     description='Format Data',
     disabled=False,
     button_style='info', # 'success', 'info', 'warning', 'danger' or ''
     tooltip='Click me',
     icon='run'
)

#########-------------------------------------------L0 FORMATTER RE-TRAIN------------------------------------------#############
def create_download_link_l0_train( df, title = "Download formatted training CSV file", filename = "formatted-data.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

def create_download_link_l0_test( df, title = "Download formatted test CSV file", filename = "formatted-data.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

def create_download_link_l0_rt_final( df, title = "Download L0 prediction CSV file", filename = "prediction-L0-RT.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())


def l0_format_rt():
#     df_pdsync_test = df_pdsync_test.rename(columns={"Deal ID": "dealid"})
    df_pdsync_rt = get_pdsync_rt.data
    df_pdsync_rt = df_pdsync_rt[['dealid', 'dealvalue','industry', 'billingtype', 'solcategory', 'projectType', 'businessimpact']]
    df_rp = get_rp.data
    df_rp = df_rp[['Year', 'Month', 'Deal ID', 'Stage', 'Weight', 'Actuals', 'Leads UW', 'Region']]
#     df_pdsync_test = df_pdsync_test.drop(columns=['Weights'], axis=1)
    df_rp = df_rp.rename(columns={"Deal ID": "dealid"})
    df_rp = df_rp.rename(columns={"Stage": "Stages"})
    df_rp = df_rp.rename(columns={"Weight": "Weights"})
    df_rp = df_rp.rename(columns={"Region": "region"})
    df_all = df_rp.merge(df_pdsync_rt.drop_duplicates(), on=['dealid','dealid'], 
                    how='left', indicator=True)
    df_all['Leads UW'] = df_all['Leads UW'].str.replace(',', '').str.replace('$', '').astype(int)
    df_all['Actuals'] = df_all['Actuals'].str.replace(',', '').str.replace('$', '').astype(int)
    dropstage = df_all['Stages'].isin(['lost', 'On Hold', 'DA', 'Deleted'])
    df_all = df_all[~dropstage]
    df_all = df_all[['Year', 'Month', 'dealid', 'Leads UW', 'Actuals','dealvalue', 'Stages',
       'Weights', 'region', 'industry', 
       'billingtype', 'solcategory', 'projectType', 'businessimpact']]
    dealvalue_eval = df_all.loc[df_all['dealvalue'].apply(lessthanzero)]
    df_all = df_all[~df_all.dealvalue.isin(dealvalue_eval.dealvalue)]
    cols2 = ['Stages','region', 'industry',
        'billingtype', 'solcategory', 'projectType', 'businessimpact']
    df_all[cols2] = df_all[cols2].astype(str)
    label_encode_columns = df_all[cols2].apply(LabelEncoder().fit_transform)
    cols1 = df_all.select_dtypes([np.number]).columns
    numerical = df_all[cols1]
    numerical['tmp'] = 1
    label_encode_columns['tmp'] = 1
    result = pd.concat([numerical, label_encode_columns], axis=1)
    result = result.drop(columns=['tmp'])
    result['Date'] = pd.to_datetime(result[['Year', 'Month']].assign(DAY=1))
    result = result.drop(columns=['Year','Month'])
    result = result[['Date',
      'dealid',                    
      'Actuals',
      'Weights',
      'Leads UW',
     'dealvalue',
     'Stages',
#     #  'PSO/Non-PSO',
      'region',
#     #  'Practice',
#     #  'sourceoflead',
       'industry',
       'billingtype',
       'solcategory',
#     #  'Business Category1',
       'projectType',
#     #  'projectextension',
       'businessimpact',
    ]]                                
                                     
    result['Date'] = pd.to_datetime(result['Date'])
    result = result.set_index(result['Date'])
    result = result.sort_index()
    result = result.drop(columns=['Date'])
#     result = result.rename(columns={"Stages": "Stage"}) 
#     result = result.rename(columns={"region": "Region"})
#     result = result.rename(columns={"Weights":"Weight"}) 
    train = result.drop(columns=['dealid'])
    train = train[['Weights', 'dealvalue','Actuals', 'Leads UW', 'Stages', 'region', 'industry', 'billingtype', 'solcategory', 'projectType', 'businessimpact']]
#     test = test.apply(pd.to_numeric)
    
    display('Displaying formatted training file below: ')
    display(train)
    display(create_download_link_l0_train(train))
    df_rt = get_test_rt.data
    df_rt = df_rt[['Year', 'Month', 'Deal ID', 'Weights', 'Stages', 'Leads UW']]
    df_rt = df_rt.rename(columns={"Deal ID": "dealid"})
#     df_rt = df_rt.rename(columns={"Region": "region"})
    df_pdsync_test = get_pdsync_rt.data
    df_pdsync_test = df_pdsync_test[['dealid', 'dealvalue','industry', 'billingtype', 'solcategory', 'projectType', 'businessimpact', 'region']]
   
    df_test = df_rt.merge(df_pdsync_test.drop_duplicates(), on=['dealid','dealid'], 
                    how='left', indicator=True)
    df_test['Leads UW'] = df_test['Leads UW'].str.replace(',', '').str.replace('$', '').astype(int)
    dropstage = df_test['Stages'].isin(['lost', 'On Hold', 'DA', 'Deleted'])
    df_test = df_test[~dropstage]
    df_test = df_test[['Year', 'Month', 'dealid', 'Leads UW','dealvalue', 'Stages',
       'Weights', 'region', 'industry', 
       'billingtype', 'solcategory', 'projectType', 'businessimpact']]
    dealvalue_eval_test = df_test.loc[df_test['dealvalue'].apply(lessthanzero)]
    df_test = df_test[~df_test.dealvalue.isin(dealvalue_eval_test.dealvalue)]
    cols2_test = ['Stages','region', 'industry',
        'billingtype', 'solcategory', 'projectType', 'businessimpact']
    df_test[cols2_test] = df_test[cols2_test].astype(str)
    label_encode_columns_test = df_test[cols2_test].apply(LabelEncoder().fit_transform)
    cols1_test = df_test.select_dtypes([np.number]).columns
    numerical_test = df_test[cols1_test]
    numerical_test['tmp'] = 1
    label_encode_columns_test['tmp'] = 1
    result_test = pd.concat([numerical_test, label_encode_columns_test], axis=1)
    result_test = result_test.drop(columns=['tmp'])
    result_test['Date'] = pd.to_datetime(result_test[['Year', 'Month']].assign(DAY=1))
    result_test = result_test.drop(columns=['Year','Month'])
    result_test = result_test[['Date',
      'dealid',                    
#       'Actuals',
      'Weights',
      'Leads UW',
     'dealvalue',
     'Stages',
#     #  'PSO/Non-PSO',
      'region',
#     #  'Practice',
#     #  'sourceoflead',
       'industry',
       'billingtype',
       'solcategory',
#     #  'Business Category1',
       'projectType',
#     #  'projectextension',
       'businessimpact',
    ]]                                
                                     
    result_test['Date'] = pd.to_datetime(result_test['Date'])
    result_test = result_test.set_index(result_test['Date'])
    result_test = result_test.sort_index()
    result_test = result_test.drop(columns=['Date'])

    result_test = result_test.drop(columns=['dealid'])
    result_test['Actuals'] = np.nan
    result_test= result_test[['Weights', 'dealvalue', 'Actuals','Leads UW', 'Stages', 'region', 'industry', 'billingtype', 'solcategory', 'projectType', 'businessimpact']]

    display('Displaying formatted testing file below: ')
    display(result_test)
    display(create_download_link_l0_test(result_test))
    x_train = train[[
     'Weights',
     'dealvalue',
     'Leads UW',
     'Stages',
     'region',
     'industry',
     'billingtype',
     'solcategory',
     'projectType',
     'businessimpact'
    ]]
    y_train = train[['Actuals']]
    x_test = result_test[[
     'Weights',
     'dealvalue',
     'Leads UW',
     'Stages',
     'region',
     'industry',
     'billingtype',
     'solcategory',
     'projectType',
     'businessimpact'
    ]]

    y_test = result_test[['Actuals']]
    
    model = XGBRegressor(learning_rate =  0.1, n_estimators=100, colsample_bytree=0.5,max_depth=1)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    pred_sum = y_pred.sum()
    text2 = 'Running Predictions for L0'
    l0pred = widgets.HTML(value = f"<b><font color='#36b38d' size=5>{text2}</b>")
    display(l0pred)
    month = df_rt['Month'].iloc[1]
    year = df_rt['Year'].iloc[1]
    print("Predicting. Please Wait..")
    time.sleep(10)
    print("Predicting. Please Wait..")
    time.sleep(10)
    print("Predicting. Please Wait..")
    time.sleep(10)
    print("Predicting. Please Wait..")
    time.sleep(10)
    print("Predicting. Please Wait..")
    time.sleep(10)
    print("Predicting. Please Wait..\n\n")
    time.sleep(5)
    print("Prediction for {}/{} is : \n".format(month,year))
    display(pred_sum)
    rmse_val = rmse(np.array(y_pred), np.array(y_test))
    print("\nTest rmse error is: " + str(rmse_val))
    rmse_val = rmse(np.array(y_pred), np.array(y_train))
    print("\nTrain rmse error is: " + str(rmse_val))
    
    y_true = y_test.sum()
    y_true, pred_sum = np.array(y_true), np.array(pred_sum)
    mape =  np.mean(np.abs((y_true - pred_sum) / y_true)) * 100
    
    mae =  np.mean(np.abs((y_true - pred_sum) / y_true))
    print("\nMAPE is: "+str(mape))
    print("\nMAE is: "+str(mae))
    df_l0_rt = pd.DataFrame(data = [[month,year,pred_sum,'nan',mae,mape]], columns=['Month', 'Year', 'Prediction', 'Actuals', 'MAE', 'MAPE'])
    display(create_download_link_l0_rt_final(df_l0_rt))
    
    
    

text3 = 'Select dates for retraining'
retrainlabel = widgets.HTML(value = f"<b><font size=4>{text3}</b>")

start_date = widgets.DatePicker(
    description='Start Date'
)
end_date = widgets.DatePicker(
       description='End Date'
)
text4 = 'Pick start date for training'
pickdates = widgets.HTML(value = f"<b><font size=2>{text4}</b>")
startend= widgets.Label("Start and end dates are from 1st to 1st, for example if training set contains data from Aug '2020 to Oct'2021 then start date will be 01-08-2021 and end date will be 01-11-2021 (1st Nov).")
lastdate= widgets.Label("The last date of the dataset in this case 31st october will also be included.")
text5 = 'Pick end date for training'
testingdate = widgets.HTML(value = f"<b><font size=2>{text5}</b>")

text6 = 'Select level of training/testing'
levlab = widgets.HTML(value = f"<b><font size=4>{text6}</b>")

text7 = 'Select prediction method'
predlab = widgets.HTML(value = f"<b><font size=4>{text7}</b>")
def evaluate(button):
    with output_box:
        selection = dataset_selector.get_interact_value()

        if (selection == "Predict with existing model"):
    #         existingdata.observe(on_change)
            get_pdsync_test_button.on_click(get_pdsync_test)
            get_test_button.on_click(get_test)
            lev_button.on_click(selectlevel)
            left_box_lev = widgets.VBox([level_selector, lev_button])
            left_box1=widgets.HBox([uploader_pdsync_test,get_pdsync_test_button])
            left_box2=widgets.HBox([uploader_test,get_test_button])
            left_box_test=widgets.VBox([label_pdsync_test,left_box1, label_test, left_box2,  levlab, left_box_lev])
            with output_box:
                    display(left_box_test) 
                               
        elif (selection == "Re-train model"):
            get_pdsync_rt_button.on_click(get_pdsync_rt)
            get_rp_button.on_click(get_rp)
            get_rt_button.on_click(get_test_rt)
            lev_button_rt.on_click(selectlevel_rt)
#             date_button.on_click(date_formatter)
            left_box_date_l0_start = widgets.VBox([start_date])
            left_box_date_l0_end = widgets.VBox([end_date])
            left_box3=widgets.HBox([uploader_pdsync_rt,get_pdsync_rt_button])
            left_box4=widgets.HBox([uploader_rp,get_rp_button])
            left_box5=widgets.HBox([uploader_test_rt,get_rt_button])
            left_box_lev_rt = widgets.VBox([level_selector_rt, lev_button_rt])
#             left_box_date = widgets.VBox([date_button])
            left_box_retrain=widgets.VBox([label_pdsync_rt,left_box3, label_rp,left_box4,label_test_rt, left_box5,retrainlabel, startend, lastdate, pickdates, start_date, testingdate, end_date,levlab, left_box_lev_rt])
            with output_box:
                    display(left_box_retrain)
#                     display(date_formatter())
                    


my_button = widgets.Button(
     description='Next',
     disabled=False,
     button_style='info', 
     tooltip='Run'
# #     icon='play'
)
# output_box = widgets.Output()
# # display(my_button, output_box)
my_button.on_click(evaluate)
left_box = widgets.VBox([predlab, dataset_selector, my_button,output_box])
widgets.HBox([left_box])
